In [46]:
from itertools import chain
import re

from htools import *

In [70]:
def hsplit(text, sep, group=True, attach=True):
    """Flexible string splitting that retains the delimiter rather, unlike
    the built-in str.split() method.
    
    Parameters
    -----------
    text: str
        The input text to be split.
    sep: str
        The delimiter to be split on.
    group: bool
        Specifies whether to group consecutive delimiters together (True),
        or to separate them (False).
    attach: bool
        Specifies whether to attach the delimiter to the string that preceeds 
        it (True), or to detach it so it appears in the output list as its own 
        item (False).
    
    Returns
    --------
    list[str]
    
    Examples
    ---------
    text = "Score -- Giants win 6-5"
    sep = '-'
    
    # Case 0.1: Delimiters are grouped together and attached to the preceding 
    word.
    >> hsplit(text, sep, group=True, attach=True)
    >> ['Score --', ' Giants win 6-', '5']
    
    # Case 0.2: Delimiters are grouped together but are detached from the 
    preceding word, instead appearing as their own item in the output list.
    >> hsplit(text, sep, group=True, attach=False)
    >> ['Score ', '--', ' Giants win 6', '-', '5']
    
    Case 1.1: Delimiters are retained and attached to the preceding string. 
    If the delimiter occurs multiple times consecutively, only the first 
    occurrence is attached, and the rest appear as individual items in the 
    output list.
    >> hsplit(text, sep, group=False, attach=True)
    >> ['Score -', '-', ' Giants win 6-', '5']
    
    # Case 1.2: Delimiters are retained but are detached from the preceding
    string.
    It appears as its own item in the output list.
    >> hsplit(text, sep, group=False, attach=False)
    >> ['Score ', '-', '-', ' Giants win 6', '-', '5']
    """
    sep_re = re.escape(sep)
    regex = f'[^{sep_re}]*{sep_re}*'        
    
    ##########################################################################
    # Case 0: Consecutive delimiters are grouped together.
    ##########################################################################
    if group:
        # Subcase 0.1
        if attach:
            return [word for word in re.findall(regex, text)][:-1]
        
        # Subcase 0.2
        else:
            return [word for word in re.split(f'({sep_re}+)', text) if word]
    
    ##########################################################################
    # Case 1: Consecutive delimiters are NOT grouped together.
    ##########################################################################
    words = text.split(sep)

    # Subcase 1.1
    if attach:
        return [word for word in re.findall(regex[:-1]+'?', text) if word]
    
    # Subcase 1.2
    return [word for word in chain(*zip(words, [sep]*len(words))) if word][:-1]

In [71]:
def sticky_split(text, sep, **kwargs):
    """Basically a functioning version of htools.hsplit with group=True and
    attach=True. Realized that doesn't work when sep contains special
    characters.
    
    Note: accepting kwargs is temporary for compatibility w/ test func
    interface.
    """
    res = []
    toks = text.split(sep)
    max_idx = len(toks) - 1
    for i, tok in enumerate(toks):
        if tok:
            if i < max_idx: tok += sep
            res.append(tok)
        elif i < max_idx:
            if res:
                res[-1] += sep
            else:
                res.append(sep)
    return res

In [72]:
def test_output(tests, sep, fn=hsplit, **kwargs):
    for test in tests:
        print(test)
        print('STANDARD:', test.split(sep))
        print('H:', fn(test, sep, **kwargs), end='\n\n')

In [73]:
text1 = 'I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.'
text2 = '*I went*to *the store and* sat next to the window*'
text3 = '**I went*to **the store and*** sat next to the window**'

texts = [text1, text2, text3]
sep = '*'

In [74]:
for text in texts:
    print(text)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
*I went*to *the store and* sat next to the window*
**I went*to **the store and*** sat next to the window**


In [75]:
test_output(texts, sep, group=True, attach=True)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
STANDARD: ['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']
H: ['I*', 'went to the store*', ' yesterday after*', ' work*', 'to*', 'see*', ' the walrus and it walked slowly*', ' over to me*', ' and sat.']

*I went*to *the store and* sat next to the window*
STANDARD: ['', 'I went', 'to ', 'the store and', ' sat next to the window', '']
H: ['*', 'I went*', 'to *', 'the store and*', ' sat next to the window*']

**I went*to **the store and*** sat next to the window**
STANDARD: ['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']
H: ['**', 'I went*', 'to **', 'the store and***', ' sat next to the window**']



In [76]:
test_output(texts, sep, fn=sticky_split)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
STANDARD: ['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']
H: ['I*', 'went to the store*', ' yesterday after*', ' work*', 'to*', 'see*', ' the walrus and it walked slowly*', ' over to me*', ' and sat.']

*I went*to *the store and* sat next to the window*
STANDARD: ['', 'I went', 'to ', 'the store and', ' sat next to the window', '']
H: ['*', 'I went*', 'to *', 'the store and*', ' sat next to the window*']

**I went*to **the store and*** sat next to the window**
STANDARD: ['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']
H: ['**', 'I went*', 'to **', 'the store and***', ' sat next to the window**']



In [77]:
# Checking that new func sticky_split is equivalent to hsplit case 1 before
# testing w/ seps w/ special chars.
def compare_split_fns(texts, sep):
    for i, text in enumerate(texts):
        hs = hsplit(text, sep)
        ss = sticky_split(text, sep)
        try:
            assert hs == ss
            print(f'{i}: same')
        except:
            print(f'{i}: DIFF')
            print('hs:', hs)
            print('ss:', ss)
        print(spacer())

In [78]:
compare_split_fns(texts, sep)

0: same

-------------------------------------------------------------------------------

1: same

-------------------------------------------------------------------------------

2: same

-------------------------------------------------------------------------------



In [79]:
texts_special = [t.replace('*', '\n\r') for t in texts]
for t in texts_special:
    print(repr(t))

'I\n\rwent to the store\n\r yesterday after\n\r work\n\rto\n\rsee\n\r the walrus and it walked slowly\n\r over to me\n\r and sat.'
'\n\rI went\n\rto \n\rthe store and\n\r sat next to the window\n\r'
'\n\r\n\rI went\n\rto \n\r\n\rthe store and\n\r\n\r\n\r sat next to the window\n\r\n\r'


In [80]:
compare_split_fns(texts_special, '\n\r')

0: DIFF
hs: ['I\n\r', 'went to the store\n\r', ' yesterday after\n\r', ' work\n\r', 'to\n\r', 'see\n\r', ' the walrus and it walked slowly\n\r']
ss: ['I\n\r', 'went to the store\n\r', ' yesterday after\n\r', ' work\n\r', 'to\n\r', 'see\n\r', ' the walrus and it walked slowly\n\r', ' over to me\n\r', ' and sat.']

-------------------------------------------------------------------------------

1: DIFF
hs: ['\n\r', 'I went\n\r', 'to \n\r', 'the store and\n\r']
ss: ['\n\r', 'I went\n\r', 'to \n\r', 'the store and\n\r', ' sat next to the window\n\r']

-------------------------------------------------------------------------------

2: DIFF
hs: ['\n\r', '\n\r', 'I went\n\r', 'to \n\r', '\n\r', 'the store and\n\r', '\n\r', '\n\r', ' sat next to the window\n\r']
ss: ['\n\r\n\r', 'I went\n\r', 'to \n\r\n\r', 'the store and\n\r\n\r\n\r', ' sat next to the window\n\r\n\r']

-------------------------------------------------------------------------------



In [51]:
sticky_split(texts_special[0])

TypeError: sticky_split() missing 1 required positional argument: 'sep'

In [7]:
test_output(texts, sep, group=True, attach=False)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
STANDARD: ['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']
H: ['I', '*', 'went to the store', '*', ' yesterday after', '*', ' work', '*', 'to', '*', 'see', '*', ' the walrus and it walked slowly', '*', ' over to me', '*', ' and sat.']

*I went*to *the store and* sat next to the window*
STANDARD: ['', 'I went', 'to ', 'the store and', ' sat next to the window', '']
H: ['*', 'I went', '*', 'to ', '*', 'the store and', '*', ' sat next to the window', '*']

**I went*to **the store and*** sat next to the window**
STANDARD: ['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']
H: ['**', 'I went', '*', 'to ', '**', 'the store and', '***', ' sat next to the window', '**']



In [8]:
test_output(texts, sep, group=False, attach=True)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
STANDARD: ['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']
H: ['I*', 'went to the store*', ' yesterday after*', ' work*', 'to*', 'see*', ' the walrus and it walked slowly*', ' over to me*', ' and sat.']

*I went*to *the store and* sat next to the window*
STANDARD: ['', 'I went', 'to ', 'the store and', ' sat next to the window', '']
H: ['*', 'I went*', 'to *', 'the store and*', ' sat next to the window*']

**I went*to **the store and*** sat next to the window**
STANDARD: ['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']
H: ['*', '*', 'I went*', 'to *', '*', 'the store and*', '*', '*', ' sat next to the window*', '*']



In [9]:
test_output(texts, sep, group=False, attach=False)

I*went to the store* yesterday after* work*to*see* the walrus and it walked slowly* over to me* and sat.
STANDARD: ['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']
H: ['I', '*', 'went to the store', '*', ' yesterday after', '*', ' work', '*', 'to', '*', 'see', '*', ' the walrus and it walked slowly', '*', ' over to me', '*', ' and sat.']

*I went*to *the store and* sat next to the window*
STANDARD: ['', 'I went', 'to ', 'the store and', ' sat next to the window', '']
H: ['*', 'I went', '*', 'to ', '*', 'the store and', '*', ' sat next to the window', '*']

**I went*to **the store and*** sat next to the window**
STANDARD: ['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']
H: ['*', '*', 'I went', '*', 'to ', '*', '*', 'the store and', '*', '*', '*', ' sat next to the window', '*', '*']



In [10]:
for text in texts:
    print(text.split(sep), end='\n\n')

['I', 'went to the store', ' yesterday after', ' work', 'to', 'see', ' the walrus and it walked slowly', ' over to me', ' and sat.']

['', 'I went', 'to ', 'the store and', ' sat next to the window', '']

['', '', 'I went', 'to ', '', 'the store and', '', '', ' sat next to the window', '', '']



In [11]:
with open('../data/lines100.txt', 'r') as f:
    lines100 = f.read()
    
with open('../data/lines5000.txt', 'r') as f:
    lines5000 = f.read()

In [12]:
tmp = hsplit(lines5000, '.', group=True, attach=True)

In [13]:
tmp = hsplit(lines5000, '.', group=False, attach=True)

In [14]:
tmp = lines5000.split('.')

In [15]:
def hpartition(text, sep):
    return text.partition(sep)